In [1]:
import numpy as np, time, math, json
from pathlib import Path

# lecture des 30 graines
seeds = [int(s) for s in Path("seeds.txt").read_text().split()]

# constantes globales
N, dt, T, steps = 30, 0.1, 200, 2000
A0, alpha_A = 1.0, 0.1
K_max, alpha_K = 5.0, 0.1
gamma_init, alpha_g = 1.5, 0.1
w1 = w2 = w3 = 1/3

In [2]:
def coherence(angles):
    return np.abs(np.mean(np.exp(1j*angles)))

def phase_entropy(angles, bins=36):
    counts, _ = np.histogram(angles, bins=bins, range=(0, 2*np.pi))
    p = counts[counts>0] / counts.sum()
    H = -np.sum(p * np.log(p))
    return H / np.log(bins)

def comfort(H, R, dRdt):
    raw = w1*H + w2*(1 - R) + w3*abs(dRdt)
    return np.tanh(raw)             # borné [-1, +1]

def G_tanh(x): return np.tanh(x)
def G_sinc(x): return np.sinc(x/np.pi)
def G_log(x):  return np.sign(x)*np.log1p(abs(x))*np.sin(x)

def G_comb(delta, w_t, w_s, w_l):
    return w_t*G_tanh(delta) + w_s*G_sinc(delta) + w_l*G_log(delta)

In [3]:
def run_one_experiment(mode:str, seed:int):
    rng = np.random.default_rng(seed)
    phi = rng.uniform(0, 2*np.pi, N)
    omega = rng.normal(2*np.pi, 0.1*2*np.pi, N)
    
    A, K, gamma = A0, 1.0, gamma_init
    prev_R = coherence(phi)
    A_trace, C_trace, R_trace = [], [], []
    
    t0 = time.perf_counter()
    for step in range(steps):
        R = coherence(phi)
        dRdt = (R - prev_R)/dt if step else 0.0
        H = phase_entropy(phi)
        C = comfort(H, R, dRdt)
        
        # cibles adaptatives
        if mode == "fps":
            A_target = A0 * (1 + C)/2
            K_target = min(1 + C, K_max)
            gamma_target = 1 + (1 - C)
            A = alpha_A*A + (1-alpha_A)*A_target
            K = alpha_K*K + (1-alpha_K)*K_target
            gamma = alpha_g*gamma + (1-alpha_g)*gamma_target
        else:   # mode témoin : constantes
            K = 1.0
        
        # poids du noyau
        w_t, w_s, w_l = abs(dRdt), H, 1-R
        w_sum = w_t + w_s + w_l
        w_t, w_s, w_l = w_t/w_sum, w_s/w_sum, w_l/w_sum
        
        E = np.mean(phi)
        for k in range(N):
            delta = E - phi[k]
            G = G_comb(delta, w_t, w_s, w_l) if mode=="fps" else 1.0
            phi[k] += dt * (omega[k] + K/N * A * np.sin(G + delta))
            phi[k] %= 2*np.pi
        
        # choc à 100 s
        if math.isclose(step*dt, 100, abs_tol=dt/2):
            idx = rng.choice(N, size=int(0.2*N), replace=False)
            phi[idx] += np.pi/2
        
        # stockage
        R_trace.append(R); A_trace.append(A); C_trace.append(C)
        prev_R = R
    
    cpu_per_step = (time.perf_counter() - t0)/steps
    
    # métriques
    area_R = np.sum(R_trace) * dt
    std_dR = np.std(np.diff(R_trace)/dt)
    mean_H = np.mean([phase_entropy(np.array(phi)) for _ in range(1)])  # approx.
    
    # références pour t_recovery
    R_ref = np.mean(R_trace[int(95/dt):int(100/dt)])
    A_ref = np.mean(A_trace[int(95/dt):int(100/dt)])
    t_recovery = next((step*dt for step,(r,a) in 
                      enumerate(zip(R_trace, A_trace), start=0)
                      if step*dt>100 and abs(r-R_ref)<0.05*R_ref 
                                        and abs(a-A_ref)<0.05*A_ref), T)
    
    effort = np.mean(np.abs(K)) if mode=="control" else \
             np.mean([abs(K*G_comb(E-phi_k,w_t,w_s,w_l)) for phi_k in phi])
    
    return dict(area_R=area_R, std_dR=std_dR, mean_H=mean_H,
                t_recovery=t_recovery, cpu_step=cpu_per_step, effort=effort)

In [4]:
import pandas as pd, os

Path("data/raw").mkdir(parents=True, exist_ok=True)

results = []
for seed in seeds:
    ctl = run_one_experiment("control", seed)
    fps = run_one_experiment("fps", seed)
    ctl["mode"] = "control"; ctl["seed"] = seed
    fps["mode"] = "fps";     fps["seed"] = seed
    results.extend([ctl, fps])

df = pd.DataFrame(results)
df.to_csv("data/raw/metrics_all.csv", index=False)
df.head()

,area_R,std_dR,mean_H,t_recovery,cpu_step,effort,mode,seed
0,32.004530,0.087991,0.794381,102.7,0.000067,1.000000,control,747319
1,32.408678,0.088737,0.823332,102.6,0.000173,0.888152,fps,747319
2,32.029349,0.076699,0.828199,100.4,0.000056,1.000000,control,711068
3,32.268189,0.076870,0.897542,100.4,0.000171,0.875510,fps,711068
4,33.601411,0.081933,0.797542,107.7,0.000055,1.000000,control,669495


In [5]:
from sklearn.utils import resample

metrics = ["area_R","std_dR","mean_H","t_recovery","cpu_step","effort"]
for m in metrics:
    diffs = df.query("mode=='fps'")[m].values - df.query("mode=='control'")[m].values
    boots = [resample(diffs).mean() for _ in range(10000)]
    ci_low, ci_high = np.percentile(boots,[2.5,97.5])
    print(f"{m:<10}  Δ={diffs.mean():.4g}   IC95%=[{ci_low:.4g},{ci_high:.4g}]")

area_R      Δ=0.1866   IC95%=[0.1347,0.242]
std_dR      Δ=0.0002924   IC95%=[0.000143,0.0004464]
mean_H      Δ=0.002944   IC95%=[-0.005979,0.01218]
t_recovery  Δ=-0.2667   IC95%=[-0.6468,0.03333]
cpu_step    Δ=0.0001158   IC95%=[0.000115,0.0001165]
effort      Δ=-0.1547   IC95%=[-0.1729,-0.1364]
